# One-vs-Rest Classifier

This notebook implements an one-vs-rest classifier that fine-tunes several BERT models to tell if a sentence contains problematic metaphors.

<div hidden>
TODO: add extend data3/data.json with better data in the same format that actually makes sense.
</div>

## Imports and Setup

In [1]:
!pip install transformers -U
!pip install sklearn -U
!pip install datasets -U
!pip install torch -U
!pip install numpy -U
!pip install evaluate -U

In [1]:
import evaluate
import numpy as np
import torch
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import (
    BertForNextSentencePrediction,
    BertTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
)
import gc

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
MODEL_NAME = "aihype_bert_fine_tune"

## Loading Dataset

In [3]:
dataset = load_dataset("json", data_files="data/pairs_unlabelled.json", field="data")
dataset

Using custom data configuration default-631b12db7123e476


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/xt0r3/.cache/huggingface/datasets/json/default-631b12db7123e476/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sen1', 'sen2', 'ans'],
        num_rows: 251461
    })
})

In [4]:
dataset["train"][0:3]

{'sen1': ['Not one savings account is currently able to keep anywhere near the pace of rising costs .',
  'People can be forgiven for losing interest and whether there is a point of tucking money away for interest paying in some circumstances 5 per cent below the rate of inflation - and this gap could grow bigger in the coming months .',
  'However , firstly for those without a savings pot whatsoever , it is important to have a rainy day fund to fall back on .'],
 'sen2': ['By Ed Magnus For Thisismoney.co.uk Published : 07:50 , 13 January 2022 | Updated : 19:04 , 13 January 2022 8 View comments Surging inflation means the outlook for savers hunting returns is bleaker than bleak .',
  'Not one savings account is currently able to keep anywhere near the pace of rising costs .',
  'People can be forgiven for losing interest and whether there is a point of tucking money away for interest paying in some circumstances 5 per cent below the rate of inflation - and this gap could grow bigger in

In [5]:
num_epochs = 30

## Preprocess Data, Create Train/Test Split

In [6]:
dataset = dataset.class_encode_column('ans')
processed_dataset = dataset["train"].train_test_split(test_size=0.2, stratify_by_column='ans')
processed_dataset

Stringifying the column:   0%|          | 0/252 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/252 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sen1', 'sen2', 'ans'],
        num_rows: 201168
    })
    test: Dataset({
        features: ['sen1', 'sen2', 'ans'],
        num_rows: 50293
    })
})

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [8]:
def preprocess_data(examples):
    return tokenizer(examples["sen1"], examples['sen2'], padding='max_length', truncation=True)

In [9]:
tokenized_dataset = processed_dataset.map(
    preprocess_data,
    remove_columns=("sen1", "sen2"),
    batched=True,
).rename_column('ans', 'next_sentence_label')

tokenized_dataset

  0%|          | 0/202 [00:00<?, ?ba/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

### Verify dataset

In [ ]:
example = tokenized_dataset['train'][3]
example.keys()

In [ ]:
tokenizer.decode(example["input_ids"])

## Load Pre-Trained Model

In [ ]:
# use_fast uses fast tokenizers backed by rust. Remove it if it causes errors
model = BertForNextSentencePrediction.from_pretrained(
    "bert-base-cased",
)

### Verify data-model interaction

In [ ]:
# forward pass
# outputs = model(
# input_ids=tokenized_dataset[labels[0]]["train"]["input_ids"][0],
# labels=tokenized_dataset[labels[0]]["train"][0]["labels"],
# )
# outputs

## Define Metrics

In [ ]:
metrics = {
    "accuracy": evaluate.load("accuracy"),
    "presicion": evaluate.load("precision"),
    "recall": evaluate.load("recall"),
    "f1": evaluate.load("f1"),
}

In [ ]:
def compute_metrics(eval_pred):
    # print(eval_pred)
    # print(list(eval_pred))
    logits, labels = eval_pred
    # print(logits, labels)
    predictions = np.argmax(logits, axis=-1)
    return {
        name: metric.compute(predictions=predictions, references=labels)
        for name, metric in metrics.items()
    }

def compute_best(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    val = metrics['f1'].compute(predictions=predictions, labels=labels)
    print(val)
    return val

## Train the Model

In [ ]:
batch_size = 1  # TODO: increase if we have more data
num_epochs = 30

In [ ]:
training_args = TrainingArguments(
    MODEL_NAME,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    report_to="all",
    label_names=['next_sentence_label'],
    load_best_model_at_end=True,
    push_to_hub=True,
)

In [ ]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(3000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,  # tokenized_dataset["train"],
    eval_dataset=small_eval_dataset,  # tokenized_dataset["test"],
    compute_metrics=compute_metrics,  # compute_metrics,
)

In [ ]:
trainer.train()

## Upload the Model

In [ ]:
# Free the memory
gc.collect()

with torch.no_grad():
    torch.cuda.empty_cache()
    
model = None
trainer = None
training_args = None
gc.collect()

In [ ]:
# agency-vs-rest/checkpoint-263: 0.75 precision, 0.85 recall
#

In [ ]:
# trainer.push_to_hub()